In [ ]:
%load_ext autoreload
%autoreload 2
from arpes.all import *
import os.path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import numpy as np
%matplotlib inline

In [ ]:
# Define directory
path = os.path.abspath(r'C:\Users\hellbrue\Documents\Bi2223_1st_beamtime')

# Different parameters used to set up scans at SLS
scan_type = '*'
direction = 'X'
phi = '*'
energy = '25eV'
polarization = '*'
temp = '*'
e_pass = '*'+'eVpass'
comment = '*'
run = '*'.zfill(4)

# Put together string for finding file
parameters = [scan_type, direction, phi, energy, polarization, temp, e_pass, comment, run]
if comment == '':
    parameters.remove(comment)
file_type = 'h5'
file_name = '_'.join(parameters)
file = '.'.join([file_name , file_type])

au_full_file = os.path.join(path, file)
file

In [ ]:
import glob
# Load all files with above specified parameters, * is a placeholder and imports all files in its place
au_files = glob.glob(au_full_file)
print(len(au_files))
au_files

In [4]:
au_raw_data_list = {}
for file in au_files:
    #data_list.append(load_data(file, location="ULTRA"))
    data = load_data(file, location="ULTRA")
    # k_data = convert_to_kspace(data.spectrum)
    file_name = file.replace(path, '')[1:-3]
    au_raw_data_list.update({file_name : data})

poly Au used to determine Fermi level. Determine Fermiedge and apply to data below

In [ ]:
au_lt = au_raw_data_list['EDC_X_P95_25eV_UNKN_LT_5eVpass_GoldRef11K_0007']
au_lt.S.plot()

In [ ]:
from arpes.analysis.general import rebin

binned = rebin(au_lt, phi=10).spectrum
binned.plot()

In [ ]:
from arpes.all import AffineBroadenedFD, broadcast_model
fermi_level = binned.sel(phi=slice(-0.2,0.2), eV=slice(-0.2,0.1))
results = broadcast_model(AffineBroadenedFD, fermi_level, "phi")

fig, ax = plt.subplots()
fermi_level.plot(ax=ax)
ax.scatter(*results.F.p("fd_center").G.to_arrays(), color="red")
ax.axhline(y=0, color='k', linestyle='--', linewidth=1)

In [ ]:
from arpes.fits.fit_models import LinearModel
lin_mod = LinearModel().guess_fit(results.F.p("fd_center"))
lin_mod.plot()
plt.show()

In [ ]:
edc = au_lt.spectrum
edge = lin_mod.eval(x=edc.phi)

corrected_edc = edc.G.shift_by(edge, "eV")

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
corrected_edc.plot(ax=axes[0], vmax=10)
edc.plot(ax=axes[1], vmax=20)
# plot vertical lines for y = 0
for ax in axes:
    ax.set_title("")
    ax.axhline(y=0, color='k', linestyle='--', linewidth=1)

In [ ]:
# Define directory
path = os.path.abspath(r'C:\Users\hellbrue\Documents\Bi2223_1st_beamtime')

# Different parameters used to set up scans at SLS
scan_type = 'FS'
direction = 'AN'
phi = 'P5'
energy = '*'
polarization = '*'
temp = '*'
e_pass = '*'+'eVpass'
comment = ''
run = '*'.zfill(4)

# Put together string for finding file
parameters = [scan_type, direction, phi, energy, polarization, temp, e_pass, comment, run]
if comment == '':
    parameters.remove(comment)
file_type = 'h5'
file_name = '_'.join(parameters)
file = '.'.join([file_name , file_type])

bi2223_full_file = os.path.join(path, file)
file

In [ ]:
import glob
# Load all files with above specified parameters, * is a placeholder and imports all files in its place
bi2223_files = glob.glob(bi2223_full_file)
print(len(bi2223_files))
bi2223_files

In [12]:
bi2223_data_list = {}
for file in bi2223_files:
    #data_list.append(load_data(file, location="ULTRA"))
    raw_data = load_data(file, location="ULTRA")
    edge = lin_mod.eval(x=raw_data.spectrum.phi)
    corrected_data = raw_data.spectrum.G.shift_by(edge, "eV")
    corrected_data.S.apply_offsets({"beta": -0.0077, "phi": -0.004})
    k_data = convert_to_kspace(corrected_data)
    file_name = file.replace(path, '')[1:-3]
    bi2223_data_list.update({file_name : k_data})

Initalize datasets

In [13]:
lt_an_p5 = bi2223_data_list['FS_AN_P5_25eV_CIRCP_LT_5eVpass_0000']
ht_an_p5 = bi2223_data_list['FS_AN_P5_25eV_CIRCP_HT_10eVpass_0000']


Overview of the LT P5 Dataset

In [ ]:
fig, ax = plt.subplots()

lt_an_p5.S.fermi_surface.plot()
lt_p5_rect_b = patches.Rectangle((-0.4, -0.58), 0.8, 0.18, linewidth=2, edgecolor='r', facecolor='none')
lt_p5_rect_t = patches.Rectangle((-0.4, 0.4), 0.8, 0.18, linewidth=2, edgecolor='w', facecolor='none')
lt_p5_rect_l = patches.Rectangle((-0.58, -0.4), 0.18, 0.8, linewidth=2, edgecolor='k', facecolor='none')
lt_p5_rect_r = patches.Rectangle((0.4, -0.4), 0.22, 0.8, linewidth=2, edgecolor='orange', facecolor='none')
ax.add_patch(lt_p5_rect_b)
ax.add_patch(lt_p5_rect_t)
ax.add_patch(lt_p5_rect_l)
ax.add_patch(lt_p5_rect_r)
plt.show()

In [ ]:
ky1 = lt_p5_rect_b.get_x()
ky2 = lt_p5_rect_b.get_x() + lt_p5_rect_b.get_width()
kx_list = np.arange(lt_p5_rect_b.get_y(), lt_p5_rect_b.get_y()+lt_p5_rect_b.get_height(), 0.02)
rows = math.ceil(len(kx_list)/3)
fig, ax = plt.subplots(rows, 3, figsize=(14, rows*4))
for i, kx_value in enumerate(kx_list):
    lt_an_p5.sel(kx=slice(kx_value, kx_value+0.02), ky=slice(ky1,ky2), eV=slice(-0.125, 0.025)).sum("kx").plot(ax=ax.ravel()[i])
    ax.ravel()[i].set_title(f"kx = {np.round(kx_value, 2)} to {np.round(kx_value+0.02, 2)}")

plt.suptitle('Red boxed area')
plt.tight_layout(rect=[0, 0.03, 1, 0.98])

In [ ]:
ky1 = lt_p5_rect_t.get_x()
ky2 = lt_p5_rect_t.get_x() + lt_p5_rect_t.get_width()
kx_list = np.arange(lt_p5_rect_t.get_y(), lt_p5_rect_t.get_y()+lt_p5_rect_t.get_height(), 0.02)
rows = math.ceil(len(kx_list)/3)
fig, ax = plt.subplots(rows, 3, figsize=(14, rows*4))
for i, kx_value in enumerate(kx_list):
    lt_an_p5.sel(kx=slice(kx_value, kx_value+0.02), ky=slice(ky1,ky2), eV=slice(-0.125, 0.025)).sum("kx").plot(ax=ax.ravel()[i])
    ax.ravel()[i].set_title(f"kx ={np.round(kx_value, 2)} to {np.round(kx_value+0.02, 2)}")

plt.suptitle('White boxed area')
plt.tight_layout(rect=[0, 0.03, 1, 0.98])

In [ ]:
ky1 = lt_p5_rect_l.get_y()
ky2 = lt_p5_rect_l.get_y() + lt_p5_rect_l.get_height()
kx_list = np.arange(lt_p5_rect_l.get_x(), lt_p5_rect_l.get_x()+lt_p5_rect_l.get_width(), 0.02)
rows = math.ceil(len(kx_list)/3)
fig, ax = plt.subplots(rows, 3, figsize=(14, rows*4))
for i, kx_value in enumerate(kx_list):
    lt_an_p5.sel(ky=slice(kx_value, kx_value+0.02), kx=slice(ky1,ky2), eV=slice(-0.125, 0.025)).sum("ky").plot(ax=ax.ravel()[i])
    ax.ravel()[i].set_title(f"ky = {np.round(kx_value, 2)} to {np.round(kx_value+0.02, 2)}")

plt.suptitle('Black boxed area')
plt.tight_layout(rect=[0, 0.03, 1, 0.98])

In [ ]:
ky1 = lt_p5_rect_r.get_y()
ky2 = lt_p5_rect_r.get_y() + lt_p5_rect_r.get_height()
kx_list = np.arange(lt_p5_rect_r.get_x(), lt_p5_rect_r.get_x()+lt_p5_rect_r.get_width(), 0.02)
rows = math.ceil(len(kx_list)/3)
fig, ax = plt.subplots(rows, 3, figsize=(14, rows*4))
for i, kx_value in enumerate(kx_list):
    lt_an_p5.sel(ky=slice(kx_value, kx_value+0.02), kx=slice(-0.4,0.4), eV=slice(-0.125, 0.025)).sum("ky").plot(ax=ax.ravel()[i])
    ax.ravel()[i].set_title(f"ky = {np.round(kx_value, 2)} to {np.round(kx_value+0.02, 2)}")

plt.suptitle('Orange boxed area')
plt.tight_layout(rect=[0, 0.03, 1, 0.98])

Overview of the HT P5 Dataset

In [ ]:
fig, ax = plt.subplots()

ht_an_p5.S.fermi_surface.plot()
ht_p5_rect_b = patches.Rectangle((-0.4, -0.58), 0.8, 0.18, linewidth=2, edgecolor='r', facecolor='none')
ht_p5_rect_t = patches.Rectangle((-0.4, 0.4), 0.8, 0.18, linewidth=2, edgecolor='w', facecolor='none')
ht_p5_rect_l = patches.Rectangle((-0.6, -0.4), 0.2, 0.8, linewidth=2, edgecolor='k', facecolor='none')
ht_p5_rect_r = patches.Rectangle((0.4, -0.4), 0.22, 0.8, linewidth=2, edgecolor='orange', facecolor='none')
ax.add_patch(ht_p5_rect_b)
ax.add_patch(ht_p5_rect_t)
ax.add_patch(ht_p5_rect_l)
ax.add_patch(ht_p5_rect_r)
plt.show()

In [ ]:
ky1 = ht_p5_rect_b.get_x()
ky2 = ht_p5_rect_b.get_x() + ht_p5_rect_b.get_width()
kx_list = np.arange(ht_p5_rect_b.get_y(), ht_p5_rect_b.get_y()+ht_p5_rect_b.get_height(), 0.02)
rows = math.ceil(len(kx_list)/3)
fig, ax = plt.subplots(rows, 3, figsize=(14, rows*4))
for i, kx_value in enumerate(kx_list):
    ht_an_p5.sel(kx=slice(kx_value, kx_value+0.02), ky=slice(ky1,ky2), eV=slice(-0.125, 0.05)).sum("kx").plot(ax=ax.ravel()[i])
    ax.ravel()[i].set_title(f"kx = {np.round(kx_value, 2)} to {np.round(kx_value+0.02, 2)}")

plt.suptitle('Red boxed area')
plt.tight_layout(rect=[0, 0.03, 1, 0.98])

In [ ]:
ky1 = ht_p5_rect_t.get_x()
ky2 = ht_p5_rect_t.get_x() + ht_p5_rect_t.get_width()
kx_list = np.arange(ht_p5_rect_t.get_y(), ht_p5_rect_t.get_y()+ht_p5_rect_t.get_height(), 0.02)
rows = math.ceil(len(kx_list)/3)
fig, ax = plt.subplots(rows, 3, figsize=(14, rows*4))
for i, kx_value in enumerate(kx_list):
    ht_an_p5.sel(kx=slice(kx_value, kx_value+0.02), ky=slice(ky1,ky2), eV=slice(-0.125, 0.05)).sum("kx").plot(ax=ax.ravel()[i])
    ax.ravel()[i].set_title(f"kx ={np.round(kx_value, 2)} to {np.round(kx_value+0.02, 2)}")

plt.suptitle('White boxed area')
plt.tight_layout(rect=[0, 0.03, 1, 0.98])

In [ ]:
ky1 = ht_p5_rect_l.get_y()
ky2 = ht_p5_rect_l.get_y() + ht_p5_rect_l.get_height()
kx_list = np.arange(ht_p5_rect_l.get_x(), ht_p5_rect_l.get_x()+ht_p5_rect_l.get_width(), 0.02)
rows = math.ceil(len(kx_list)/3)
fig, ax = plt.subplots(rows, 3, figsize=(14, rows*4))
for i, kx_value in enumerate(kx_list):
    ht_an_p5.sel(ky=slice(kx_value, kx_value+0.02), kx=slice(ky1,ky2), eV=slice(-0.125, 0.05)).sum("ky").plot(ax=ax.ravel()[i])
    ax.ravel()[i].set_title(f"ky = {np.round(kx_value, 2)} to {np.round(kx_value+0.02, 2)}")

plt.suptitle('Black boxed area')
plt.tight_layout(rect=[0, 0.03, 1, 0.98])

In [ ]:
ky1 = ht_p5_rect_r.get_y()
ky2 = ht_p5_rect_r.get_y() + ht_p5_rect_r.get_height()
kx_list = np.arange(ht_p5_rect_r.get_x(), ht_p5_rect_r.get_x()+ht_p5_rect_r.get_width(), 0.02)
rows = math.ceil(len(kx_list)/3)
fig, ax = plt.subplots(rows, 3, figsize=(14, rows*4))
for i, kx_value in enumerate(kx_list):
    ht_an_p5.sel(ky=slice(kx_value, kx_value+0.02), kx=slice(-0.4,0.4), eV=slice(-0.125, 0.05)).sum("ky").plot(ax=ax.ravel()[i])
    ax.ravel()[i].set_title(f"ky = {np.round(kx_value, 2)} to {np.round(kx_value+0.02, 2)}")

plt.suptitle('Orange boxed area')
plt.tight_layout(rect=[0, 0.03, 1, 0.98])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
lt_an_p5.sel(eV=slice(-0.02,0.02)).sum("eV").plot(ax=ax, cmap='viridis')

# plt.ylim(-0.56, 0)
# plt.xlim(-0, 0.56)
plt.title('')
plt.ylabel('$k_y$ ($\AA^{-1}$)', fontsize=18)
plt.xlabel('$k_x$ ($\AA^{-1}$)', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=16)
# plt.yticks(np.arange(-0.4, 0.5, 0.2))
# plt.xticks(np.arange(-0.4, 0.5, 0.2))
ax.collections[0].colorbar.remove()
cbar = plt.colorbar(ax.collections[0], ax=ax)
cbar.set_label('Intensity', fontsize=16, labelpad=5)
cbar.set_ticks([])
# set x and y axis to be equal
plt.gca().set_aspect('equal', adjustable='box')
plt.show()